In [ ]:
from elasticsearch import Elasticsearch
import pandas as pd
# Create the client instance
client = Elasticsearch("http://localhost:9200")

# Successful response!
client.info()

In [ ]:
recipes = pd.read_json('data/recipes_cleaned_1.json')

In [ ]:
def create_index(client, index_name):
    # Define the index settings and mappings
    index_settings = {
        "settings": {
            "analysis": {
                "filter": {
                    "arabic_stemmer": {
                        "type": "stemmer",
                        "language": "arabic"
                    },
                    "arabic_stop": {
                        "type": "stop",
                        "stopwords": "_arabic_"
                    }
                },
                "analyzer": {
                    "rebuilt_arabic": {
                        "type": "custom",
                        "tokenizer": "standard",
                        "filter": ["lowercase", "decimal_digit", "arabic_stop", "arabic_normalization", "arabic_stemmer"]
                    }
                }
            }
        },
        "mappings": {
            "properties": {
                "Id": {
                    "type": "long"
                },
                "RecipeTitle": {
                    "type": "text",
                    "analyzer": "rebuilt_arabic",
                    "search_analyzer": "rebuilt_arabic"
                },
                "Ingredients": {
                    "type": "text",
                    "analyzer": "rebuilt_arabic",
                    "search_analyzer": "rebuilt_arabic"
                },
                "Tags": {
                    "type": "text",
                    "analyzer": "rebuilt_arabic",
                    "search_analyzer": "rebuilt_arabic"
                },
                "Steps": {
                    "type": "text",
                    "analyzer": "rebuilt_arabic",
                    "search_analyzer": "rebuilt_arabic"
                }
            }
        }
    }

    # Create the index with the specified settings and mappings
    create_index_response = client.indices.create(index=index_name, body=index_settings)
    return create_index_response
# Call the function to create the index
response = create_index(client=client, index_name="recipes")
print(response)

In [ ]:
def add_recipes(client, index, recipes):
    #add to recipes id column from its index
    recipes['Id'] = recipes.index
    # Convert the dataframe to a dictionary
    recipes_dict = recipes.to_dict("records")
    for recipe in recipes_dict:
        # Index the dictionary of each recipe into Elasticsearch
        resp = client.index(index=index, document=recipe)
        if resp['result'] != 'created':
            print(resp)


In [ ]:
add_recipes(client, 'recipes', recipes)